# Testing the implementation of the SigGPDE signature kernel and its gradients
***

For SigGPDE we implemented a recent kernel trick to approximate the signature kernel in `gpsig.kernels_pde`. We provide both a Cuda and a Cython implementation. 

In this notebook:
- we validate the implementation of the SigGPDE kernel by comparing the results to those obtained with the *truncated signature kernel* trick from GPSig (see `gpsig.kernels`)
- we time the gradients computation

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import sys
sys.path.append('..') 

import numpy as np
import matplotlib.pyplot as plt
import gpsig
import tensorflow as tf
import time

We start by generating some random data

In [2]:
num_examples = 300
len_examples = 20
num_features = 5
input_dim = len_examples*num_features
X = np.random.randn(num_examples, len_examples, num_features)
X = X.cumsum(axis=1)
X/=np.max(X)

***
### Validating the implementation of the SigPDE signature kernel (forward)


##### Computing the SigPDE signature kernel with the Cython operator

In [3]:
order = 2
kern_cpu = gpsig.kernels_pde.UntruncSignatureKernel(input_dim, num_features, implementation='cython',order=order)
K_pde_cpu = kern_cpu.compute_Kdiag(X.reshape([num_examples, -1]))

##### Computing the SigGPDE signature kernel with the Cuda operator

In [4]:
order = 2
kern_gpu = gpsig.kernels_pde.UntruncSignatureKernel(input_dim, num_features, implementation='gpu_op',order=order)
K_pde_gpu = kern_gpu.compute_Kdiag(X.reshape([num_examples, -1]))

##### Computing the truncated signature kernel

We need to take a high number of levels to ensure we have a good approximation of the signature kernel

In [5]:
num_levels = 9
kern_trunc = gpsig.kernels.SignatureLinear(input_dim, num_features, num_levels, order=num_levels, normalization=False)
K_trunc = kern_trunc.compute_Kdiag(X.reshape([num_examples, -1]))

##### Comparing the results

In [6]:
K_diff = K_pde_cpu - K_trunc
print(np.allclose(K_pde_cpu, K_trunc,rtol=1e-4))

True


In [7]:
K_diff = K_pde_gpu - K_trunc
print(np.allclose(K_pde_cpu, K_trunc,rtol=1e-4))

True


***
### Computing the gradients of the SigPDE signature kernel (backward)


##### Computing with the Cython operator

In [8]:
with tf.Session(graph=tf.Graph()) as sess: 
    init = tf.initialize_all_variables()
    sess.run(init)
    X_ = tf.constant(X)    
    kern = gpsig.kernels_pde.UntruncSignatureKernel(input_dim, num_features, implementation='cython',order=order)
    
    t = time.time()
    res = kern.Kdiag(tf.reshape(X_,[-1,input_dim]))
    grad = tf.gradients(res, X_,unconnected_gradients=tf.UnconnectedGradients.NONE)
    grad = sess.run(grad)
    
    print('time Cython: ',  np.round(time.time()-t,3), 's')

time Cython:  0.231 s


##### Computing with the Cuda operator

In [10]:
with tf.Session(graph=tf.Graph()) as sess: 
    init = tf.initialize_all_variables()
    sess.run(init)
    X_ = tf.constant(X)    
    kern = gpsig.kernels_pde.UntruncSignatureKernel(input_dim, num_features, implementation='gpu_op',order=order)

    t = time.time()
    res = kern.Kdiag(tf.reshape(X_,[-1,input_dim]))
    grad = tf.gradients(res, X_,unconnected_gradients=tf.UnconnectedGradients.NONE)
    grad = sess.run(grad)

    print('time Cuda: ',  np.round(time.time()-t,3), 's')

time Cuda:  0.318 s


##### Computing with the truncated kernel

In [12]:
with tf.Session(graph=tf.Graph()) as sess:   
    init = tf.initialize_all_variables()
    sess.run(init)
    kern_trunc = gpsig.kernels.SignatureLinear(input_dim, num_features, num_levels, order=num_levels, normalization=False)
    X_ = tf.constant(X)

    t = time.time()    
    res = kern_trunc.Kdiag(tf.reshape(X_,[-1,input_dim]))
    grad_trunc = tf.gradients(res, X_,unconnected_gradients=tf.UnconnectedGradients.NONE)
    grad_trunc = sess.run(grad_trunc)[0]
    print('time Truncated: ', np.round(time.time()-t,3), 's')

time Truncated:  4.054 s
